## REI602M Machine Learning - Project 1
### Due: *Friday* 20.04.2018

**Objectives**: Allstate Claims Severity

**Name**: Þór Tómasarson, **email: ** tht33@hi.is, **collaborators:** Eggert Jón Magnússon

In [17]:
#### Preprocess the data ####
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv('data/train.csv', delimiter=',', index_col='id')
X_train, y_train = train.loc[:, train.columns != 'loss'], train['loss']

X_train.describe()    # Is there need to transform the 'loss' column? (mean: 3037.3, std: 2904.1, min: 0.67, max: 121012.25)

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
count,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000
mean,0.493861,0.507188,0.498918,0.491812,0.487428,0.490945,0.484970,0.486437,0.485506,0.498066,0.493511,0.493150,0.493138,0.495717
std,0.187640,0.207202,0.202105,0.211292,0.209027,0.205273,0.178450,0.199370,0.181660,0.185877,0.209737,0.209427,0.212777,0.222488
min,0.000016,0.001149,0.002634,0.176921,0.281143,0.012683,0.069503,0.236880,0.000080,0.000000,0.035321,0.036232,0.000228,0.179722
25%,0.346090,0.358319,0.336963,0.327354,0.281143,0.336105,0.350175,0.312800,0.358970,0.364580,0.310961,0.311661,0.315758,0.294610
50%,0.475784,0.555782,0.527991,0.452887,0.422268,0.440945,0.438285,0.441060,0.441450,0.461190,0.457203,0.462286,0.363547,0.407403
75%,0.623912,0.681761,0.634224,0.652072,0.643315,0.655021,0.591045,0.623580,0.566820,0.614590,0.678924,0.675759,0.689974,0.724623
max,0.984975,0.862654,0.944251,0.954297,0.983674,0.997162,1.000000,0.980200,0.995400,0.994980,0.998742,0.998484,0.988494,0.844848


No need to transform any of the continuous variables. They are all on the interval [0, 1] with reasonable standard deviation.

In [18]:
# One-hot transform
X_train = pd.get_dummies(X_train)

# Do a train - test split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.30)

In [19]:
# TODO
# línulega/ridge/lasso/Huber aðhvarfsgreiningu, trjálíkön eða tauganet

In [20]:
import time
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

# Fit estimators
ESTIMATORS = {
    "Extra trees": ExtraTreesRegressor(n_estimators=10, max_features=32,
                                       random_state=0),
    "K-nn": KNeighborsRegressor(),
    "Linear regression": LinearRegression(),
    "Ridge": RidgeCV(),
}

y_test_predict = dict()
for name, estimator in ESTIMATORS.items():
    t_0 = time.time()
    print(name)
    estimator.fit(X_train, y_train)
    
    t_1 = time.time()
    print(f'Time elapsed for fitting of model {t_1 - t_0:.3f} sec')
    y_test_predict[name] = estimator.predict(X_test)
    print(f'Time elapsed for prediction {time.time() - t_1:.3f} sec')

Extra trees
Time elapsed for fitting of model 34.959 sec
Time elapsed for prediction 1.342 sec
K-nn
Time elapsed for fitting of model 119.266 sec
Time elapsed for prediction 17775.757 sec
Linear regression
Time elapsed for fitting of model 30.304 sec
Time elapsed for prediction 0.485 sec
Ridge
Time elapsed for fitting of model 62.882 sec
Time elapsed for prediction 0.497 sec


In [21]:
for name, y_hat in y_test_predict.items():
    error = np.sum(np.abs(y_hat - y_test)) / len(y_test)
    print(f'{name}, error: {error:.3f}')

Extra trees, error: 1308.083
K-nn, error: 1415.175
Linear regression, error: 25835782568.925
Ridge, error: 1306.844
